In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from tqdm.notebook import tqdm
from torch.cuda.amp import autocast, GradScaler

In [2]:
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
testing_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)
training_set, validation_set = torch.utils.data.random_split(training_set, [55000, 5000])
#training_set, validation_set = torch.utils.data.random_split(training_set, [540000,60000])

In [3]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=128, shuffle=True, num_workers=4,  pin_memory=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
testing_loader = torch.utils.data.DataLoader(testing_set, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)

In [4]:
class VGG11(nn.Module):

  def __init__(self):
    super(VGG11, self).__init__()
    self.layers = nn.Sequential(

        nn.Conv2d(1, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        #nn.Dropout(0.5),

        nn.MaxPool2d(2, 2),

        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        #nn.Dropout(0.5),

        nn.MaxPool2d(2, 2),

        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        #nn.Dropout(0.5),

        nn.MaxPool2d(2, 2),

        nn.Conv2d(256, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        #nn.Dropout(0.5),

        nn.MaxPool2d(2, 2),

        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        #nn.Dropout(0.5),

        nn.MaxPool2d(2, 2),
        nn.Flatten(),


        # Classifier
        nn.Linear(512*7*7, 4096),
        nn.BatchNorm1d(4096),
        nn.ReLU(),
        nn.Dropout(0.5),

        nn.Linear(4096, 4096),
        nn.BatchNorm1d(4096),
        nn.ReLU(),
        nn.Dropout(0.5),

        nn.Linear(4096, 10)

    )


    self.loss = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.Adam(self.parameters(), lr=0.0005)
    self.scheduler = None

  def forward(self, x):
    return self.layers(x)

  def train_model(self, training_loader, validation_loader, epoch):
    self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=0.001, steps_per_epoch=len(training_loader), epochs=epoch)
    scaler = GradScaler()

    self.to('cuda')


    training_loss = []
    training_accuracy = []
    validation_loss = []
    validation_accuracy = []

    for i in range(epoch):
        true = 0
        total = 0
        running_loss = 0
        for images, labels in tqdm(training_loader, desc=f"Training Epoch {i}"):
            self.train()
            images = images.to('cuda')
            labels = labels.to('cuda')

            # model training
            self.optimizer.zero_grad()
            with autocast():
              pred = self.forward(images)
              loss = self.loss(pred, labels)
            scaler.scale(loss).backward()
            scaler.step(self.optimizer)
            scaler.update()
            self.scheduler.step()

            # preformance eval
            running_loss += loss
            pred_actual = torch.argmax(pred, dim=-1)
            true += (pred_actual == labels).sum().item()
            total += labels.size(0)

        # training set eval
        training_loss.append(running_loss/len(training_loader))
        training_accuracy.append(true / total *100)

        # validation set eval
        val_loss, val_accuracy = self.test_model(validation_loader, f"Validating Epoch {i}")
        validation_loss.append(val_loss)
        validation_accuracy.append(val_accuracy)

        print(f"Training Loss: {training_loss[-1]}, Training Accuracy: {training_accuracy[-1]}, Validation Loss: {validation_loss[-1]}, Validation Accuracy: {validation_accuracy[-1]}")
        model.save_model("vgg11_mnist.pth")

    #return training_loss, training_accuracy, validation_loss, validation_accuracy


  def test_model(self, testing_loader, desc):
      self.eval()
      self.to('cuda')
      true = 0
      total= 0
      loss = 0
      with torch.no_grad():
        for images, labels in tqdm(testing_loader, desc = desc):
            images = images.to('cuda')
            labels = labels.to('cuda')
            pred = self.forward(images)
            pred_actual = torch.argmax(pred, dim=-1)
            loss += self.loss(pred, labels)
            true += (pred_actual == labels).sum().item()
            total += labels.size(0)
        accuracy = true / total * 100
        return loss/len(testing_loader), accuracy


  def save_model(self, path):
      torch.save(self.state_dict(), path)
      print(f"Model parameters saved to {path}")

  def load_model(self, path):
      self.load_state_dict(torch.load(path))
      self.to('cuda')
      print(f"Model parameters loaded from {path}")



In [5]:
print(f"CUDA Available: {torch.cuda.is_available()}")
model = VGG11().to('cuda')
model.train_model(training_loader, validation_loader, 15)



CUDA Available: True


Training Epoch 0:   0%|          | 0/430 [00:00<?, ?it/s]

C:\Users\Jack\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Validating Epoch 0:   0%|          | 0/40 [00:00<?, ?it/s]

Training Loss: 0.38002869486808777, Training Accuracy: 86.14, Validation Loss: 0.27231594920158386, Validation Accuracy: 89.72
Model parameters saved to vgg11_mnist.pth


...

Validating Epoch 13:   0%|          | 0/40 [00:00<?, ?it/s]

Training Loss: 0.00125404994469136, Training Accuracy: 99.97818181818182, Validation Loss: 0.24736976623535156, Validation Accuracy: 95.28
Model parameters saved to vgg11_mnist.pth


Training Epoch 14:   0%|          | 0/430 [00:00<?, ?it/s]

Validating Epoch 14:   0%|          | 0/40 [00:00<?, ?it/s]

Training Loss: 0.0010372466640546918, Training Accuracy: 99.97818181818182, Validation Loss: 0.25207948684692383, Validation Accuracy: 95.22
Model parameters saved to vgg11_mnist.pth


In [7]:
#model.load_model("vgg11_mnist.pth")
test_loss, test_accuracy = model.test_model(testing_loader, "Testing Preformance ")
print(f"Training Loss: {test_loss}, Training Accuracy: {test_accuracy}")




Testing Preformance :   0%|          | 0/79 [00:00<?, ?it/s]

Training Loss: 0.28056347370147705, Training Accuracy: 94.46
